[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-durand/propedeutico_python/blob/main/notebooks/7_SQL.ipynb)

# Propedéutico a programación con Python.

**Verano 2023, por el Centro de Ciencia de Datos, EGobiernoyTP.**

## Sesión 7: SQL básico

1. Introducción
    * Bases de datos
    * Lenguaje SQL
2. Establecer conexiones con las bases de datos
    * sqlite 
    * psycopg2 
3. Acciones básicas con SQL
    * Instrucciones desde Python
    * Obtener información con SQL
    * Modificar información con SQL
    
---
# 1 Introducción

## Bases de datos

Las bases de datos relacionales es otra manera en la que se puede almacenar información.


Generalmente, las bases de datos de esta naturaleza se guardan en servidores (computadoras en la nube) dedicados exclusivamente para el mantenimiento y manejo de datos tabulares (columna-renglón). Las bases de datos relacionales son de gran ayuda sobre todo cuando se manejan grandes cantidades de datos con una estructura muy amigable.

RDBMS stands for Relational Database Management System.

continuar . . .


---

## Lenguaje SQL

Un lenguaje estándar que se utiliza para interactuar con estas bases es SQL (Structured Query Language). Con SQL puedes almacenar, manipular y recuperar información en la tabla o sobre ella. Específicamente, además de crear bases de datos te permite hacer las siguientes actividades con una base de datos:

* hacer búsquedas 
* obtener datos
* insertar datos
* actualizar datos
* borrar elementos (renglones/columnas/tablas)
* crear tablas 
* automatizar procedimientos almacenados
* definir permisos sobre tablas y procedimientos


# 2 Establecer conexiones con las bases de datos

Algunos paquetes para interactuar entre python y bases de datos relacionales son `psycopg2` y `sqlite`, ambos utilizan SQL. El que utilizarás más durante la maestría es `psycopg2` y lo usarás para interactuar con servidores. Sin embargo, para enseñarte el funcionamiento de SQL aquí usaremos bases de datos locales con ayuda de `sqlite`.

---

## Establecer conexión con sqlite

Primero cargamos la librería:

In [ ]:
import sqlite3

Para fines de la sesión de hoy utilizamos una base de datos local para mostrar cómo puedes cargar datos con sql. 
La base se llama `flights` y está en la carpeta datos:

In [ ]:
gdrv_file = '../datos/flights.db'

Después generas una conexión con la base de datos dicha conexión permite a Python saber en dónde se encuentra la base con la que estarás trabajando. Si no existiera una base con ese nombre entonces `sqlite3` la crearía implícitamente.

In [ ]:
con = sqlite3.connect(gdrv_file)
con

## Establecer conexión a través de psycopg2

En el caso de psycopg2, para establecer la conexión a bases de datos (del tipo PSQL) necesitas las siguientes líneas de código, así como la información que se enumera:


```
# Importas paquete
import psycopg2

# Estableces conexión: 
psconn = psycopg2.connect(host="mouse.db.elephantsql.com", # 1. Dirección de la base
                          port = 5432,                     # 2. Puerto de conexión
                          database="base_mouse",           # 3. Nombre de la base
                          user="usuario-n",                # 4. Nombre de usuario
                          password="your-password")        # 5. Contraseña
                          

```


La información numerada se asigna y proporciona generalmente por el administrador/dueño de la base de datos. Esto es para tener mayor seguridad y control de la información.

---


# 3 Obtener información con SQL

Una vez que tienes una conexión entre Python y la base de datos, puedes utilizar instrucción escritas en el lenguaje SQL para interactuar con ella. 

Una instrucción (o _query_) es simplemente una oración que puede tener distintos comandos (o palabras clave) que realizan acciones sobre una base de datos o una tabla.

Un ejemplo es el siguiente:

`SELECT * FROM airlines;`

en este caso las palabras clave están escritas con mayúsculas y hacen lo siguiente:

- `SELECT` : extrae información
- `*` : selecciona todas las columnas
- `FROM` : de 
- `airlines` : tabla airlines
- `;` : termina instrucción

Las palabras clave no distinguen entre mayúsculas y minúsculas. Por ejemplo, `select` es equivalente a `SELECT`. Pero es recomendable escribir en mayúsculas pues el estándar en varios proyectos. * El punto y coma indica que una instrucción acaba y permite separarlo de otros y ejecutarlos en una misma llamada.

En la siguiente celda guardamos la instrucción en una cadena de texto de Python para poder ejecutarla a través de Python. Las instrucciones se pueden ejecutar a través de Python -como lo mostraremos en este tutorial- o a través de una terminal sin requerir Python.


In [ ]:
q = '''
    SELECT 
        * 
    FROM 
        airlines
        ;
    '''

Así, en la siguiente celda, con ``q`` obtendremos todos los datos de la tabla `airlines` a este notebook. Lo haremos a través de la conexión recién creada, y para que el formato de las interacciones con la tabla sea estético utilizaremos `pandas` (se puede cargar con un formato más simple utilizando sólo `sqlite`).

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_sql(q, con)
df

Con la siguiente instrucción en lugar de obtener todas las columnas de la tabla (`*`) podemos obtener sólo algunas columnas (`name`, `country`, `active`):

In [ ]:
q = '''
    SELECT 
        name, country, active
    FROM 
        airlines
        ;
    '''
df = pd.read_sql(q, con)
df

La columna `active` de la tabla indica si la aerolínea se encuentra activa. Podemos hacer la misma búsqueda anterior pero obteniendo solamente aerolíneas activas, con ayuda el comando `WHERE`:

In [ ]:
q = '''
    SELECT 
        name, country, active
    FROM 
        airlines
    WHERE
        active="Y"
    '''
df = pd.read_sql(q, con)
df

Otro comando útil es `ORDER BY`. Naturalmente, permite ordenar la tabla de salida respecto a alguna columna de interés. Podemos por ejemplo ordenar la salida anterior alfabéticamente respecto al nombre de la aerolínea:

In [ ]:
q = '''
    SELECT 
        name, country, active
    FROM 
        airlines
    WHERE
        active="Y"
    ORDER BY
        name
    '''
df = pd.read_sql(q, con)
df.head(20)

En la celda anterior visualizamos únicamente los primeros 20 renglones con ayuda del método `head()` de `pandas`. Podemos restringirnos a los primeros "n" renglones directamente desde SQL con el comando `LIMIT`:

In [ ]:
q = '''
    SELECT 
        name, country, active
    FROM 
        airlines
    WHERE
        active="Y"
    ORDER BY
        name
    LIMIT 
        20
    '''
df = pd.read_sql(q, con)
df

En la condición de `WHERE` de la celda anterior utilizamos la operación de comparación "igual" (`=`). SQL acepta distintos operadores de comparación y lógicos, algunos otros son:

* `>`: mayor que
* `<`: menor que 
* `!=`: distinto de
* `AND`: verdadero si dos expresiones son verdaderas.
* `IN`: verdadero si la expresión está en una lista de expresiones
* `NOT`: regresa el valor inverso de la expresión
* `OR`: verdadero si alguna expresión es verdadera
* `BETWEEN`: verdadero si la expresión está dentro de un rango de valores

---

Enseguida recapitulamos los comandos que hemos revisado y que son de los más útiles de SQL: 

* `SELECT` - extrae información de la tabla.
* `FROM`- indica de cuál tabla se obtendrá la información.
* `WHERE` - filtra la salida de la instrucción basándose en una condición.
* `ORDER BY` - ordena la tabla de salida.
* `LIMIT`- se limita a las primeros 'n' renglones de la tabla de salida.

otros comandos muy útiles son:

* `GROUP BY` - agrupa renglones que tienen los mismos valores en cierta(s) columna(s)
en renglones que las resumen. Normalmente se utiliza en conjunto con los comandos `COUNT`, `AVG`, `SUM`, `MIN` o `MAX`; éstos últimos se llaman funciones agregadas pues hacen operaciones en *grupos* de renglones (puede ser en todos los renglones) de columnas numéricas.

A continuación vemos un ejemplo de aplicación de `GROUP BY` con `COUNT`:

In [ ]:
q = '''
    SELECT 
        country, COUNT(name), COUNT(active)
    FROM 
        airlines
    GROUP BY
        country
    '''
df = pd.read_sql(q, con)
df

Otros comandos utilizados a menudo son:

* `HAVING` -  es análogo a WHERE pero en lugar de actuar sobre condiciones que incluyen directamente a las columnas (por ejemplo, WHERE `active="Y"`), actúa sobre condiciones de columnas previamete procesadas (por ejemplo, HAVING `COUNT(active) > 5`).
* `JOIN` - es similar al comando `merge` de `pandas`, es decir, sirve para unir tablas basándose en los valores de de cierta columna.

---

Como hemos visto en los ejemplos anteriores, podemos ir agregando comandos a una instrucción. Al utilizar el lenguaje SQL es bueno tener mente que el orden en el que se escriben las intrucciones es diferente del orden en el que se ejecutan:

| Orden de escritura | Orden de ejecución | 
|--------------------|--------------------|
| SELECT             | FROM               | 
| FROM               | JOIN               |  
| JOIN               | WHERE              |  
| WHERE              | GROUP BY           | 
| GROUP BY           | HAVING             | 
| HAVING             | SELECT             | 
| ORDER BY           | ORDER BY           | 
| LIMIT              | LIMIT              | 




## Modificar bases de datos

`UPDATE` - updates data in a database
`DELETE` - deletes data from a database
`INSERT INTO` - inserts new data into a database


`CREATE DATABASE` - creates a new database
`ALTER DATABASE` - modifies a database
`CREATE TABLE` - creates a new table
`ALTER TABLE` - modifies a table
`DROP TABLE` - deletes a table
`CREATE INDEX` - creates an index (search key)
`DROP INDEX` - deletes an index